In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [31]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path="drive/My Drive/Colab Notebooks/ML/MachineLearning/NLP/Chatbot1/intents.json"
with open(path,'r') as f:
  intents = json.load(f)

FILE="drive/My Drive/Colab Notebooks/ML/MachineLearning/NLP/Chatbot1/"+"data.pth"
data = torch.load(FILE)

input_size  = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words   = data['all_words']
tags        = data['tags']
model_state = data['model_state']

class Nltk_utils:
  
  def __init__(self,sentence):
    self.sentence = sentence

    #tokenizing
    self.tokenized_words = self.tokenize()

    #stemming
    self.stemmed_words = self.tokenized_words
    for i in range(len(self.stemmed_words)):
      self.stemmed_words[i] = self.stem(self.stemmed_words[i])

  def tokenize(self):
    return nltk.word_tokenize(self.sentence);

  def stem(self,word):
    pstem = PorterStemmer()
    return pstem.stem(word.lower())

  #for next tutorial
  def bag_of_words(self,tokenized_sentence,all_words):    #have an array of all_words
    tokenized_sentence = [self.stem(w) for w in (tokenized_sentence)]
    bag = np.zeros(len(all_words),dtype=np.float32)

    for idx,w in enumerate(all_words):
      if w in tokenized_sentence:
        bag[idx] = 1.0

    return bag

# Model Creating in PyTorch
class NeuralNetFF(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNetFF,self).__init__()

    self.l1 = nn.Linear(input_size,hidden_size)
    self.l2 = nn.Linear(hidden_size,hidden_size)
    self.l3 = nn.Linear(hidden_size,num_classes)
    self.relu = nn.ReLU()
  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    # no activation , no sofmax
    return out
model = NeuralNetFF(input_size,hidden_size,output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Mama"
print("let's chat !! Type 'quit' to exit ...")

import random

while True:
  sentence = input("You : ")
  if sentence == "quit":
    break
  nlp = Nltk_utils(sentence)

  bagsss = nlp.bag_of_words(nlp.tokenized_words,all_words)
  X = bagsss.reshape(1,bagsss.shape[0])
  X = torch.from_numpy(X)

  output = model(X)

  _, predicted = torch.max(output,dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output,dim=1)
  prob = probs[0][predicted.item()]

  print(tags)
  print(tag)
  print(prob)

  if prob> 0.70 :
    for intent in intents["intents"]:
      if(tag == intent["tag"]):
        print(f"{bot_name} : {random.choice(intent['responses'])}")
  else:
    print(f'{bot_name} : I do not understant. Can you be more specific?')


  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
let's chat !! Type 'quit' to exit ...
You : Hi there
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
greeting
tensor(0.9997, grad_fn=<SelectBackward>)
Mama : Hi there, how can I help?
You : I need some cofe
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
greeting
tensor(0.5030, grad_fn=<SelectBackward>)
Mama : I do not understant. Can you be more specific?
You : I want to buy some coffe
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
greeting
tensor(0.5030, grad_fn=<SelectBackward>)
Mama : I do not understant. Can you be more specific?
You : What is your item?
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
items
tensor(0.8103, grad_fn=<SelectBackward>)
Mama : We have coffee and tea
You : Send some coffe to my home
['delivery', 'funny', 'goodbye', 'greeting', 'items'